In [10]:
import math
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from AutoEncoder import AutoEncoder
from utils import toTorchTensers, NormalizeData

base_url = r'.'

data_url = rf'{base_url}/data/Data.xlsx'
data_egypt = pd.read_excel(data_url, sheet_name='Egypt')
data_vietnam = pd.read_excel(data_url, sheet_name='Vietnam')

device = 'cpu'
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)


In [11]:
# data
DATA = data_egypt
DATA = NormalizeData(DATA)
DATA = DATA.to_numpy()

np.random.seed(1)
np.random.shuffle(DATA)
split_point = math.floor(len(DATA) * 0.75)

train_x, train_y = DATA[:split_point, 1:], DATA[:split_point, 0:1]
test_x, test_y = DATA[split_point:, 1:], DATA[split_point:, 0:1]

[train_x, train_y, test_x, test_y] = toTorchTensers(
    train_x, train_y, test_x, test_y,
    device=device
)


In [12]:
gen_min = 3
gen_max = 12
gen_count = 3

population_count = 4
generation_count = 100

# 
def generate_individual():
  individual = []
  for i in range(gen_count):
    individual.append(random.randint(gen_min, gen_max))

  return individual


def generate_population():
    population = []
    for i in range(population_count):
        population.append(generate_individual())

    return population


def calc_fitnesses(population):
  fitnesses = []
  for i in range(population_count):
    model = AutoEncoder(population[i])
    model.fit(train_x, train_x, epoches=1600, batch_size=600, progress=False)
    fitnesses.append(model.current_loss)
  return fitnesses
  

def get_best(fitnesses):
  return np.argmin(fitnesses)


def get_bests(fitnesses, count):
    fitnesses = fitnesses[:]
    bests = []

    for i in range(count):
      b = np.argmin(fitnesses)
      bests.append(b)

      fitnesses.pop(i)
    
    return bests


def breed(p1, p2):
    gene1 = random.randint(0, gen_count-1)
    gene2 = random.randint(0, gen_count-1)

    new = p1[:]

    new[gene1] = p2[gene1]
    new[gene2] = p2[gene2]

    return new


In [13]:
# # # # #
population = generate_population()
global_best = None
global_best_fitness = None

for i in range(0, generation_count):
  print(f"generation {i+1}")
  fitnesses = calc_fitnesses(population)
  best = get_best(fitnesses)
  print(f"Best: {fitnesses[best]}")
  if global_best == None:
      global_best = population[best]
      global_best_fitness = fitnesses[best]
  elif global_best_fitness > fitnesses[best]:
      global_best = population[best]
      global_best_fitness = fitnesses[best]

  best_two = get_bests(fitnesses, 2)
  
  population = [
    population[best_two[0]],
    population[best_two[1]],
    breed(population[best_two[0]], population[best_two[1]]),
    breed(population[best_two[1]], population[best_two[0]]),
  ]

print(f"global best: {global_best_fitness}")
print(global_best)


generation 1
✅ training ended ,final loss: 0.00942987, time: 8.1s
✅ training ended ,final loss: 0.01507597, time: 7.5s
✅ training ended ,final loss: 0.03940317, time: 8.2s
✅ training ended ,final loss: 0.01330483, time: 7.6s
Best: 0.00942987110465765
generation 2
✅ training ended ,final loss: 0.01387687, time: 8.5s
✅ training ended ,final loss: 0.01206211, time: 8.2s
✅ training ended ,final loss: 0.00824314, time: 8.4s
✅ training ended ,final loss: 0.01288559, time: 7.9s
Best: 0.00824313797056675
generation 3
✅ training ended ,final loss: 0.00607302, time: 7.9s
✅ training ended ,final loss: 0.03965230, time: 7.8s
✅ training ended ,final loss: 0.01401429, time: 8.0s
✅ training ended ,final loss: 0.01794920, time: 8.0s
Best: 0.00607302226126194
generation 4
✅ training ended ,final loss: 0.01175040, time: 8.2s
✅ training ended ,final loss: 0.01070335, time: 7.7s
✅ training ended ,final loss: 0.00809591, time: 7.9s
✅ training ended ,final loss: 0.01282744, time: 7.8s
Best: 0.00809591170400

KeyboardInterrupt: 

In [17]:
global_best
# global_best_fitness


[7, 12, 12]